# Day 4 - Lab 2: Generating a CI/CD Pipeline (Solution)

**Objective:** Use an LLM to generate all necessary configuration files to create an automated Continuous Integration (CI) pipeline for the FastAPI application using Docker and GitHub Actions.

**Introduction:**
This solution notebook provides the complete prompts for generating the `requirements.txt`, `Dockerfile`, and GitHub Actions workflow files. It demonstrates how to prompt for specific, structured configuration-as-code artifacts.

## Step 1: Setup

## Step 2: The Challenges - Solutions

### Challenge 1 (Foundational): Generating a `requirements.txt`

**Explanation:**
This prompt is an analysis task. We provide the application's source code as context and ask the LLM to parse the import statements to identify external libraries. This is much faster and less error-prone than manually creating the file.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path
try:
    # This works when running as a script
    project_root = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))
except NameError:
    # This works when running in an interactive environment (like a notebook)
    # We go up two levels from the notebook's directory to the project root.
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
requirements_prompt = f"""You are a Python dependency analysis tool. Analyze the following Python code and generate a `requirements.txt` file listing all the external libraries imported.

Include versions for key libraries like fastapi, uvicorn, sqlalchemy, and pydantic.

--- PYTHON CODE ---
{app_code}
--- END CODE ---
"""

print("--- Generating requirements.txt ---")
if app_code:
    requirements_content = get_completion(requirements_prompt, client, model_name, api_provider)
    print(requirements_content)
    save_artifact(requirements_content, "requirements.txt")
else:
    print("Skipping requirements generation because app code is missing.")

### Challenge 2 (Intermediate): Generating a `Dockerfile`

**Explanation:**
This prompt asks for a specific, best-practice `Dockerfile`. By explicitly requesting a "multi-stage" build, a "slim" base image, and a "non-root user," we guide the LLM to generate a configuration that is both efficient and secure, saving the developer from having to remember these important details.

In [ ]:
dockerfile_prompt = """You are a DevOps expert specializing in containerization. Generate a best-practice, multi-stage `Dockerfile` for a production Python FastAPI application.

The Dockerfile must:
1.  Use `python:3.11-slim` as the base image.
2.  Have a `builder` stage to install dependencies from `requirements.txt` into a virtual environment.
3.  The final stage should copy the application code and the virtual environment from the builder.
4.  Create and run as a non-root user named `appuser` for security.
5.  Expose port 8000.
6.  The final `CMD` should run the application using `uvicorn`.
"""

print("--- Generating Dockerfile ---")
dockerfile_content = get_completion(dockerfile_prompt, client, model_name, api_provider)
print(dockerfile_content)

if dockerfile_content:
    save_artifact(dockerfile_content, "Dockerfile")

### Challenge 3 (Advanced): Generating the GitHub Actions Workflow

**Explanation:**
This prompt generates a complete CI workflow in YAML format. We are very specific about the required structure: the `on` trigger, the `jobs` definition, and the `steps` within the job. This level of detail ensures the LLM generates a syntactically correct and logically complete workflow file that is ready to be committed to the repository.

In [ ]:
ci_workflow_prompt = """You are a CI/CD specialist. Generate a complete GitHub Actions workflow file named `ci.yml` for a Python FastAPI project.

The workflow must:
- Be named 'Build and Test'.
- Trigger on every `push` to the `main` branch.
- Define one job named `build-and-test` that runs on `ubuntu-latest`.
- The job must have the following sequential steps:
  1. `actions/checkout@v3` to check out the repository code.
  2. `actions/setup-python@v4` to set up Python 3.11.
  3. A step to install dependencies using pip from `requirements.txt`.
  4. A step to run the test suite using `pytest`.
"""

print("--- Generating GitHub Actions Workflow ---")
ci_workflow_content = get_completion(ci_workflow_prompt, client, model_name, api_provider)
print(ci_workflow_content)

if ci_workflow_content:
    # Clean up the response if it's wrapped in markdown fences
    if '```' in ci_workflow_content:
        ci_workflow_content = ci_workflow_content.split('```')[1].lstrip('yaml').strip()
    save_artifact(ci_workflow_content, ".github/workflows/ci.yml")